In [ ]:
!pip install sentence-transformers
!pip install qdrant-client

In [ ]:
# pip install qdrant-client
# pip install sentence-transformers

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

In [ ]:
#QdrantSetup
client = QdrantClient(
    url="URL_DE_SEU_CLUSTER",
    api_key="SUA_API_KEY",
)

In [ ]:
titulo = "pump device"
resumo = "the invention has a housing a pump section formed of a drive gear unit and a driven gear unit a main flow channel through which oil pressure is applied to the driven gear unit in a discharge volume reduction direction a first branching flow channel through which oil pressure that assists oil pressure from the main flow channel is applied a second branching flow channel through which oil pressure is applied to the driven gear unit in a discharge increase direction a first flow channel control section a second flow channel control section and a spring that elastically urges the driven gear unit in a discharge increase direction the first flow channel control section and the second flow channel control section can perform switching control in accordance with each increase or decrease of engine revolutions and in pressure"


In [ ]:
# Carrega a patente

patent_ds = pd.json_normalize([{
        "Abstract": resumo, 
        "Title": titulo
    }])

patent_ds

In [ ]:
index_name = "CLUSTER_NAME" # Name of your Cluster

dimensions_768 = 768 #Size of embeddings
model_768 = SentenceTransformer('all-distilroberta-v1') #Loads the pre-trained SBERT model

dimensions_384 = 384 #Size of embeddings
model_384 = SentenceTransformer('all-MiniLM-L6-v2') #Loads the pre-trained SBERT model

dimensions = dimensions_384
model = model_384

In [ ]:
def histogram(classes:list, k:int):
    result = {}
    ctr=1
    for value, key in sorted(((classes.count(e), e) for e in set(classes)), reverse=True):
        if (ctr > k): break
        result[key] = value
        ctr+=1
    return result

In [ ]:
def process_result(accuracy_dict, k, n, type):
    index = "{}-{}-{}".format(k,n,type)
    if (index in accuracy_dict):
        accuracy_dict[index] = accuracy_dict.get(index) + 1
    else:
        accuracy_dict[index] = 1

In [ ]:
def get_process_result(accuracy_dict, k, n, type):
    index = "{}-{}-{}".format(k,n,type)
    if (index in accuracy_dict):
        return accuracy_dict[index]
    else:
        return 0

In [ ]:
def print_process_result(accuracy_dict, k_list, n_list):
    for k in k_list:
        for n in n_list:
            positive = get_process_result(accuracy_dict, k, n, 'positive')
            negative = get_process_result(accuracy_dict, k, n, 'negative')
            accuracy = positive / (positive + negative)
            print("k={} - n={} - Positive: {} - Negative: {} - "
                "Accuracy: {} ".format(k,n,positive,negative,accuracy))

In [ ]:
def transform_process_result(accuracy_dict, k_list, n_list):
    matrix = np.zeros((len(k_list), len(n_list)))
    i = j = 0
    for k in k_list:
        j=0
        for n in n_list:
            positive = get_process_result(accuracy_dict, k, n, 'positive')
            negative = get_process_result(accuracy_dict, k, n, 'negative')
            accuracy = positive / (positive + negative)
            matrix[i][j] = accuracy
            j+=1
        i+=1
    return matrix

In [ ]:
#Method to approximate search
def search(vector):
    hits = client.search(
        collection_name=index_name,
        query_vector=vector,
        limit=100
    )
    return hits

In [ ]:
#Performs queries to group returned patent subclasses for each input patent
print("Index name: ",index_name)
k_list = [1,2,3,4,5,6,7,8,9,10]
n_list = [10,25,50,75,100]
accuracy_dict = {}
ctr_hit = 0
hits = 0
positive = negative = 0
subclass_list = []
hit_list = []
ctr_queries = 0
for index, row in patent_ds.iterrows():
    ctr_queries+=1
    #if (ctr_queries > 2000): break
    embedding = model.encode(row.Title+" "+row.Abstract).tolist()

    hits = search(embedding)

    hit_list.clear()
    hits_count=0
    for hit in hits:
        hit_list.append(hit.payload['subclass_labels'])
        #if (hits_count <= 20): print(str(hit.score) +" - "+hit.payload['subclass_labels'])
        hits_count+=1

    k=10
    n=50
    ctr_hit = 0
    for subclass in hit_list:
        ctr_hit+=1
        if (ctr_hit > n): break
        subclass_list.append(subclass)

    histogram_res = histogram(subclass_list, k)
    #print('k: '+str(k)+ ' - n: '+str(n))
    print(histogram_res)
    subclass_list.clear()